In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.spatial.distance import cdist


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
broadcast_video = 'broadcast.mp4'
tacticam_video = 'tacticam.mp4'
model_path = 'best.pt'
confidence_threshold = 0.4

In [5]:
model = YOLO(model_path)

In [6]:
def detect_players(frame):
    result = model(frame)
    detections = []
    for r in result[0].boxes:
        if r.conf > confidence_threshold and r.cls == 2:
            x1, y1, x2, y2 = map(int, r.xyxy[0].tolist())
            detections.append({'bbox': (x1, y1, x2, y2), 'confidence': float(r.conf)})
    return detections

In [7]:
def extract_appearance_feature(frame, bbox):
    x1, y1, x2, y2 = bbox
    crop = frame[y1:y2, x1:x2]
    crop = cv2.resize(crop, (32, 64))
    hist = cv2.calcHist([crop], [0, 1, 2], None, [8, 8, 8],
                        [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [8]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    features_per_frame = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        detections = detect_players(frame)
        features = []
        for detection in detections:
            feat = extract_appearance_feature(frame, detection['bbox'])
            features.append({'bbox': detection['bbox'], 'feature': feat})
        features_per_frame.append(features)
    cap.release()
    return features_per_frame

In [14]:
def match_players(broadcast_frame_features, tacticam_frame_features):
    if not broadcast_frame_features or not tacticam_frame_features:
        return {}
    broad_feats = np.array([p['feature'] for p in broadcast_frame_features])
    tactic_feats = np.array([p['feature'] for p in tacticam_frame_features])
    dists = cdist(broad_feats, tactic_feats, metric='cosine')
    mapping = {}
    for i, row in enumerate(dists):
        j = np.argmin(row)
        mapping[i] = j
    return mapping

In [15]:
def main():
    broadcast_features_all_frames = process_video(broadcast_video)
    tacticam_features_all_frames = process_video(tacticam_video)

    all_frames_mapping = []
    for frame_idx, (broadcast_frame_features, tacticam_frame_features) in enumerate(zip(broadcast_features_all_frames, tacticam_features_all_frames)):
        mapping = match_players(broadcast_frame_features, tacticam_frame_features)
        all_frames_mapping.append(mapping)

    print("Player mapping for all frames:")
    for frame_idx, mapping in enumerate(all_frames_mapping):
        print(f"Frame {frame_idx}:")
        if mapping:
            for broadcast_idx, tacticam_idx in mapping.items():
                print(f"  Broadcast player {broadcast_idx} is matched with Tacticam player {tacticam_idx}")
        else:
            print("  No players were detected or matched in this frame.")

In [16]:
if __name__ == "__main__":
    main()


0: 384x640 3 players, 66.0ms
Speed: 4.1ms preprocess, 66.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 65.9ms
Speed: 3.2ms preprocess, 65.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 53.9ms
Speed: 2.7ms preprocess, 53.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 37.8ms
Speed: 2.9ms preprocess, 37.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 38.1ms
Speed: 3.0ms preprocess, 38.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 38.5ms
Speed: 3.1ms preprocess, 38.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 38.2ms
Speed: 3.1ms preprocess, 38.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 14 players, 1 referee, 37.1ms
Speed: 3.